

Author: Ronny F. Pacheco Date: Nov 2024 Copyright: © 2024 Ronny Pacheco License: MIT License

MIT License

Copyright (c) 2024 Ronny Pacheco

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


# In/Out files

In files:
- ../data/software_result/Last_Data.csv

Out files:

- data/sider_df_v2.csv


# Needed modules

In [3]:
import pandas as pd
import os
import pickle

In [4]:
# https://kioku-space.com/en/jupyter-skip-execution/
from IPython.core.magic import register_cell_magic # type: ignore


@register_cell_magic
def skip(line, cell):
    print('Skipping cell')
    if line and cell:
        pass
    return

# Pickle save

In [5]:
%%skip
# =============================================================================
# main function
# =============================================================================
def data_save_load(option):
    """
    This function is used to save or load data for the jupyter notebook
    """
    path_folder = "ipynb_db"  # Folder to save variables
    os.makedirs(path_folder, exist_ok=True)  # Create folder if not exist
    notebook_name = '1.prepare_data'
    path_file = os.path.join(path_folder, f"{notebook_name}.variables.pkl") # Path to save the variables

    if option == "save":
        with open(path_file, "wb") as pickle_file:
            dict_variables = { # 'var_name' : 'real_var'

            }
            pickle.dump(dict_variables, pickle_file)
    elif option == "load":
        with open(path_file, "rb") as pickle_file:
            variables = pickle.load(pickle_file)
        # Now load the variables
        for pickle_key, pickle_value in variables.items():
            print(f"* Loading variable: {pickle_key}")
            globals()[pickle_key] = pickle_value
# =============================================================================
# Call the function
# =============================================================================
data_save_load(option="load")

Skipping cell


# Load data

In [6]:
sider_df = pd.read_csv('../data/software_result/Last_Data.csv', header=0, sep=',')
print(sider_df.shape)
print(sider_df.dtypes)
sider_df.head()

(4804, 14)
qseqid      float64
sseqid       object
pident      float64
length        int64
qstart      float64
qend        float64
sstart        int64
send          int64
evalue      float64
bitscore    float64
qlen        float64
slen        float64
sstrand      object
sseq         object
dtype: object


,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
0,NaN,LmjF.01,NaN,253,NaN,NaN,24536,24284,NaN,NaN,NaN,NaN,minus,CCCTCCCCGTGGGCCAGTGTGAGGAGCCGGCTGCGATGCGCTCGAG...
1,NaN,LmjF.01,NaN,597,NaN,NaN,25200,24604,NaN,NaN,NaN,NaN,minus,TATGACAGTTTGCCGCCCAGCGGCGCTGCGGTGTACCGCTGGCCAG...
2,NaN,LmjF.01,NaN,935,NaN,NaN,36401,35467,NaN,NaN,NaN,NaN,minus,GCCACGAATTCTTCGTGGGCCTCTCCGTCTCTCCGTGCCATCTGCT...
3,NaN,LmjF.01,NaN,812,NaN,NaN,40541,39730,NaN,NaN,NaN,NaN,minus,GCCCCAGCCCCTTGGCTGGCATGGACGGAGATGGACGATGAAGACA...
4,NaN,LmjF.01,NaN,949,NaN,NaN,55542,54594,NaN,NaN,NaN,NaN,minus,GCAGCGGATTTCCATAGGCTTCTCTCAGCTCCAGGCAATCGGTTGA...


# Prepare data

## Extract needed columns

In [10]:
# Get needed columns only and rename columns.
sider_df = sider_df[['sseqid', 'length', 'sstart', 'send', 'sstrand', 'sseq']]
sider_df = sider_df.rename(
    columns= {
        'sseqid': 'chrom',
        'sstart': 'sider_start',
        'send': 'sider_end',
        'sstrand': 'sider_strand',
        'sseq': 'sider_seq'
    }
)
print(sider_df.shape)
print(sider_df.dtypes)
sider_df.head()

(4804, 6)
chrom           object
length           int64
sider_start      int64
sider_end        int64
sider_strand    object
sider_seq       object
dtype: object


,chrom,length,sider_start,sider_end,sider_strand,sider_seq
0,LmjF.01,253,24536,24284,minus,CCCTCCCCGTGGGCCAGTGTGAGGAGCCGGCTGCGATGCGCTCGAG...
1,LmjF.01,597,25200,24604,minus,TATGACAGTTTGCCGCCCAGCGGCGCTGCGGTGTACCGCTGGCCAG...
2,LmjF.01,935,36401,35467,minus,GCCACGAATTCTTCGTGGGCCTCTCCGTCTCTCCGTGCCATCTGCT...
3,LmjF.01,812,40541,39730,minus,GCCCCAGCCCCTTGGCTGGCATGGACGGAGATGGACGATGAAGACA...
4,LmjF.01,949,55542,54594,minus,GCAGCGGATTTCCATAGGCTTCTCTCAGCTCCAGGCAATCGGTTGA...


## Fail proof: Start > End ?

In [18]:
# Check where rows 'sider_start' > 'sider_end'
print(sider_df[sider_df['sider_start'] > sider_df['sider_end']].shape)
print(sider_df[sider_df['sider_start'] > sider_df['sider_end']]['sider_strand'].value_counts())
sider_df[sider_df['sider_start'] > sider_df['sider_end']].head()

(2874, 6)
sider_strand
minus    2874
Name: count, dtype: int64


,chrom,length,sider_start,sider_end,sider_strand,sider_seq
0,LmjF.01,253,24536,24284,minus,CCCTCCCCGTGGGCCAGTGTGAGGAGCCGGCTGCGATGCGCTCGAG...
1,LmjF.01,597,25200,24604,minus,TATGACAGTTTGCCGCCCAGCGGCGCTGCGGTGTACCGCTGGCCAG...
2,LmjF.01,935,36401,35467,minus,GCCACGAATTCTTCGTGGGCCTCTCCGTCTCTCCGTGCCATCTGCT...
3,LmjF.01,812,40541,39730,minus,GCCCCAGCCCCTTGGCTGGCATGGACGGAGATGGACGATGAAGACA...
4,LmjF.01,949,55542,54594,minus,GCAGCGGATTTCCATAGGCTTCTCTCAGCTCCAGGCAATCGGTTGA...


In [21]:
# Change it so 'sider_start' < 'sider_end'
sider_df.loc[
    sider_df[sider_df['sider_start'] > sider_df['sider_end']].index,
    ['sider_start', 'sider_end']
] = sider_df.loc[
    sider_df[sider_df['sider_start'] > sider_df['sider_end']].index,
    ['sider_end', 'sider_start']
].values

# Check result
# Check where rows 'sider_start' > 'sider_end'
print(sider_df[sider_df['sider_start'] > sider_df['sider_end']].shape)
print(sider_df[sider_df['sider_start'] > sider_df['sider_end']]['sider_strand'].value_counts())
sider_df[sider_df['sider_start'] > sider_df['sider_end']].head()

(0, 6)
Series([], Name: count, dtype: int64)


,chrom,length,sider_start,sider_end,sider_strand,sider_seq


In [23]:
# Check if the number of 'sider_start' < 'sider_end' are the same as the total length of 'sider_df'
print(sider_df.shape)
print(sider_df[sider_df['sider_start'] < sider_df['sider_end']].shape)

(4804, 6)
(4804, 6)


# Save data

In [26]:
# Load folder
folder_name = 'data'
os.makedirs(folder_name, exist_ok=True)

# Save data
file_path = os.path.join(folder_name, 'sider_df_v2.csv')
sider_df.to_csv(file_path, index=False, header=True, sep=',')
print(f"File saved to: {file_path}")

File saved to: data/sider_df_v2.csv
